In [1]:
import os
import logging
import pandas as pd
import mysql.connector

In [3]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,  # Change to your MySQL server
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE  # Optional: specify database
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [4]:
# Create a cursor object
cursor = conn.cursor()

In [5]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [8]:
query="SELECT * FROM Promotion;"
promotion_df = extract(cursor, query)

In [9]:
promotion_df

,PromotionID,PromotionName,Type,StartDate,EndDate
0,1,Summer Sale,Discount,2023-06-01,2023-09-01
1,2,Holiday Special,Bundle,2022-12-01,2023-12-31
2,3,Back to School,Discount,2023-08-01,2023-09-15
3,4,Spring Clearance,Clearance,2023-03-01,2023-04-30
4,5,Weekend Flash Sale,Flash,2023-07-15,2023-07-16
5,6,Black Friday,Discount,2023-11-24,2023-11-25
6,7,Cyber Monday,Discount,2023-11-27,2023-11-27
7,8,New Year Special,Bundle,2023-12-31,2024-01-01
8,9,Valentine's Day Offer,Discount,2023-02-01,2023-02-14
9,10,Easter Sale,Discount,2023-04-01,2023-04-15


In [10]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [11]:
import psycopg2
from psycopg2 import sql

In [12]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

print(PG_HOST, PG_USERNAME, PG_PASSWORD, PG_DATABASE)

172.18.0.2 biruni P@ssw0rd walmart_dwh


In [13]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,  # or your database host
    'port': 5432          # default PostgreSQL port
}

In [14]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [15]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [19]:
query = sql.SQL(f"""
    INSERT INTO dimpromotion (promotionbk, promotionname, type, startdate, enddate)
    VALUES (%s, %s, %s, %s, %s);
""")

In [20]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading employee: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [21]:
load(cursor, query, employees_df)

Data Loaded successfully!


In [22]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
